In [1]:
import numpy as np
import matplotlib.pyplot as plt
from beam_mechanics2_script import simulate_beam, m_dir_from_psi

In [ ]:
def make_ref_constant(theta_deg):
    """Constant setpoint in degrees. Returns a callable f(t)->(theta_ref_rad, theta_ref_dot_rad_s)."""
    theta_rad = np.deg2rad(theta_deg)
    return lambda t: (theta_rad, 0.0)

def make_ref_sine(A_deg, f_hz, offset_deg=0.0, phase_deg=0.0):
    """Sine reference θ_ref(t)=offset + A·sin(2πft+φ)."""
    A = np.deg2rad(A_deg)
    w = 2*np.pi*f_hz
    off = np.deg2rad(offset_deg)
    ph = np.deg2rad(phase_deg)
    return lambda t: (off + A*np.sin(w*t + ph), A*w*np.cos(w*t + ph))

In [ ]:
theta_ref_deg = 20.0
theta_ref_rad = np.deg2rad(theta_ref_deg)

psi_min_deg = locals().get('psi_min_deg', -90.0)
psi_max_deg = locals().get('psi_max_deg',  90.0)
psi_rate_limit_deg = locals().get('psi_rate_limit_deg', 3.0)
alpha_d = locals().get('alpha_d', 0.2)
sign_g  = locals().get('sign_g', 1.0)

Kp_pd = locals().get('Kp_pd', 1.5)
Kd_pd = locals().get('Kd_pd', 0.05)

In [ ]:
def run_PD():
    # logs
    t_hist, th_hist_deg, thref_hist_deg, psi_hist_deg, u_hist_deg = [], [], [], [], []
    # state
    psi_pd_deg = 0.0
    prev_err_deg = 0.0
    derr_filt = 0.0

    N = int(Tsim/dt)
    for k in range(N):
        t = k*dt
        # plant measure (rad)
        theta_meas_rad, _, _, _ = simulate_beam(
            p_vec, psi_pd_deg,
            L=L, A_val=A_val, E_val=E_val, I_val=I_val,
            MU0=MU0, MAGNET_M=MAGNET_M,
            m_line_of_s=m_line_of_s,
            s_steps=200
        )
        theta_meas_deg_PD = np.degrees(theta_meas_rad)

        # PD error/filter
        err_deg_PD  = theta_ref_deg - theta_meas_deg_PD
        derr_raw    = (err_deg_PD - prev_err_deg) / dt
        derr_filt   = (1 - alpha_d) * derr_filt + alpha_d * derr_raw
        prev_err_deg = err_deg_PD

        # PD control → Δψ, rate-limit & clamp ψ
        delta_psi_deg_PD = sign_g * (Kp_pd * err_deg_PD + Kd_pd * derr_filt)
        delta_psi_deg_PD = float(np.clip(delta_psi_deg_PD, -psi_rate_limit_deg, psi_rate_limit_deg))
        psi_pd_deg = float(np.clip(psi_pd_deg + delta_psi_deg_PD, psi_min_deg, psi_max_deg))

        # logs
        t_hist.append(t)
        th_hist_deg.append(theta_meas_deg_PD)
        thref_hist_deg.append(theta_ref_deg)
        psi_hist_deg.append(psi_pd_deg)
        u_hist_deg.append(delta_psi_deg_PD/dt)  # approx ψ_dot in deg/s

    return dict(
        t=np.array(t_hist),
        theta_deg=np.array(th_hist_deg),
        theta_ref_deg=np.array(thref_hist_deg),
        psi_deg=np.array(psi_hist_deg),
        u_deg_s=np.array(u_hist_deg),
    )